In [30]:
import numpy as np
import pyuvdata
import newcal
import importlib
importlib.reload(newcal)
from newcal import calibration_optimization
importlib.reload(calibration_optimization)
from newcal import calibration_wrappers
importlib.reload(calibration_wrappers)
import multiprocessing

In [27]:
data = pyuvdata.UVData()
data.read("/Users/ruby/Astro/test_calibration_Jul2023/20230309_225134_73MHz.ms")

In [28]:
model = pyuvdata.UVData()
model.read("/Users/ruby/Astro/test_calibration_Jul2023/20230309_225134_73MHz_model.uvfits")

In [31]:
(
    gains_init,
    Nants,
    Nbls,
    Ntimes,
    Nfreqs,
    N_feed_pols,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
    antenna_names,
) = calibration_wrappers.uvdata_calibration_setup(
    data,
    model,
    gain_init_calfile=None,
    gain_init_stddev=0.0,
    N_feed_pols=2,
    min_cal_baseline=30,
)

divide by zero encountered in true_divide


In [ ]:
gains_fit = calibration_wrappers.calibration_per_pol(
    gains_init,
    Nants,
    Nbls,
    Nfreqs,
    2,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
    xtol=1e-3,
    parallel=False,
    verbose=True,
)

Optimization terminated successfully.
         Current function value: 817666880600.477539
         Iterations: 148
         Function evaluations: 255
         Gradient evaluations: 255
         Hessian evaluations: 148
Optimization terminated successfully.
Optimization time: 3.5857065955797833 minutes
Optimization terminated successfully.
         Current function value: 651399913559.489258
         Iterations: 30
         Function evaluations: 36
         Gradient evaluations: 36
         Hessian evaluations: 30
Optimization terminated successfully.
Optimization time: 0.625172233581543 minutes


In [29]:
gains_fit = calibration.run_calibration_optimization_per_pol(
    gains_init,
    Nants,
    Nbls,
    Nfreqs,
    2,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
    100,
    xtol=1e-5,
    verbose=False,
)

17636940115717.46
4859945175126.358
2254064497988.7554
1607356335501.2666
1022069575273.1179
963298390953.4988
925170935499.8705
923020179550.0942
918988215045.6342
912108023565.0875
905061122713.8375
904896422056.4917
904581951916.6926
904026665448.1554
903519090570.4712
903361355808.7039
903349015172.2039
903324873459.7657
903245806766.0519
903197124197.1476
903108579111.9224
902969324342.6909
902966379177.8519
902960526707.0319
902939052223.4832
902895142405.8789
902894388642.955
902892885307.6031
902887242604.4254
902874855315.6973
902868260057.6547
902867949519.2258
902867325982.3937
902864906915.3914
902856452395.8671
902855588975.9225
902853873526.7928
902847589128.5001
902847167126.3552
902846330322.4729
902843297477.1697
902842994601.7915
902842385725.2808
902840008924.994
902831461123.1492
902830536447.5216
902828745118.8705
902823130927.1855
902822135062.454
902820373546.9204
902817802048.2974
902817704328.9982
902817507338.8649
902816727434.6533
902813736938.5034
9028039240

902663881577.0557
902663881352.5446
902663880463.6904
902663877055.3077
902663865774.3701
902663865440.2694
902663865419.415
902663865378.3282
902663865234.7638
902663865220.7518
902663865192.7794
902663865087.5258
902663864825.6506
902663863922.9866
902663863795.6268
902663863785.8022
902663863765.9877
902663863687.3486
902663863382.707
902663862322.7812
902663862210.8336
902663862026.4575
902663862020.5349
902663862008.596
902663861961.329
902663861780.0991
902663861180.585
902663861094.4133
902663861087.9857
902663861075.0813
902663861025.0485
902663860850.2811
902663860832.2507
902663860795.9512
902663860653.1864
902663860121.0762
902663860091.7477
902663860032.8428
902663859803.9995
902663858997.0381
902663858917.2854
902663858911.4812
902663858899.8219
902663858854.4882
902663858694.0118
902663858677.7292
902663858644.9265
902663858515.4741
902663858025.813
902663858004.3483
902663857961.2179
902663857793.2651
902663857194.5477
902663857138.1306
902663857133.62
902663857124.5293


902663847995.5563
902663847993.2473
902663847985.4843
902663847984.2429
902663847982.5304
902663847979.3113
902663847979.0652
902663847978.5714
902663847976.6595
902663847970.034
902663847969.0156
902663847967.0636
902663847965.7289
902663847963.3137
902663847963.1705
902663847962.8826
902663847961.7548
902663847957.6329
902663847957.2253
902663847956.42
902663847953.5612
902663847953.3062
902663847952.795
902663847950.8317
902663847944.2694
902663847943.1581
902663847941.7325
902663847941.6838
902663847941.5858
902663847941.1974
902663847939.7158
902663847934.9277
902663847933.969
902663847932.0801
902663847925.4758
902663847924.6188
902663847922.914
902663847916.6246
902663847916.083
902663847915.14
902663847913.2933
902663847907.0737
902663847906.1425
902663847904.7268
902663847902.0321
902663847901.8875
902663847901.5966
902663847900.4587
902663847896.3198
902663847895.9263
902663847895.1394
902663847892.1514
902663847891.4664
902663847890.1233
902663847885.5552
902663847884.7559
9

KeyboardInterrupt: 

In [ ]:
np.max(visibility_weights[:, :, 0, 0,])

In [ ]:
freq_ind = 0
pol_ind = 0
gains_init_flattened = np.stack(
    (
        np.real(gains_init[:, freq_ind, pol_ind]),
        np.imag(gains_init[:, freq_ind, pol_ind]),
    ),
    axis=0,
).flatten()

cost = calibration.cost_function_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, freq_ind, pol_ind],
    data_visibilities[:, :, freq_ind, pol_ind],
    visibility_weights[:, :, freq_ind, pol_ind],
    gains_exp_mat_1,
    gains_exp_mat_2,
    0.01,
)
jac = calibration.jacobian_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, freq_ind, pol_ind],
    data_visibilities[:, :, freq_ind, pol_ind],
    visibility_weights[:, :, freq_ind, pol_ind],
    gains_exp_mat_1,
    gains_exp_mat_2,
    0.01,
)
hess = calibration.hessian_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, freq_ind, pol_ind],
    data_visibilities[:, :, freq_ind, pol_ind],
    visibility_weights[:, :, freq_ind, pol_ind],
    gains_exp_mat_1,
    gains_exp_mat_2,
    0.01,
)

In [ ]:
print(cost)

In [ ]:
print(jac)
print(hess)

In [ ]:
np.where(hess - np.conj(hess).T != 0)

In [ ]:
np.linalg.eigvals(np.conj(hess.T))

In [ ]:
hess - np.conj(hess).T

In [7]:
freq_ind = 0
pol_ind = 0
antenna_weights = np.sum(
    np.matmul(
        gains_exp_mat_1.T,
        visibility_weights[
            :,
            :,
            freq_ind,
            pol_ind,
        ].T,
    )
    + np.matmul(
        gains_exp_mat_2.T,
        visibility_weights[
            :,
            :,
            freq_ind,
            pol_ind,
        ].T,
    ),
    axis=1,
)

In [8]:
print(antenna_weights)

[  0.   0.   0.   0.   0.   0.   0.   0. 247.   0.   0.   0.   0.   0.
   0.   0.   0. 247.   0. 247. 247. 247.   0. 247.   0. 247.   0.   0.
 247.   0. 247.   0.   0. 247. 247.   0.   0.   0.   0.   0.   0.   0.
   0.   0. 247.   0. 247.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247. 247. 247. 247. 247. 247.   0. 247. 247. 247. 247. 247. 247.
 247. 247. 247. 247. 247. 247.   0. 247.   0. 247.   0.   0. 247. 247.
 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247.   0. 247. 247. 247. 247. 247. 247. 247. 247.   0.   0.   0.
   0.   0. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247.   0. 247. 247. 247. 247. 247. 247. 247. 247.   0.   0. 247.
 247.   0.   0. 247.   0. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247. 247. 247. 247. 247. 247. 247. 247. 247.   0. 247. 247. 247.
 247. 247. 247. 247. 247. 247.   0. 247.   0. 247. 247. 247. 247. 247.
 247. 

In [20]:
args_list = [
    (np.full((3,4),5), np.full((3,4),5)),
    (np.full((5,4),10), np.full((5,4),2))]
with multiprocessing.Pool() as pool:
    result = pool.starmap(calibration.test_function, args_list)
print(result)

[array([[25, 25, 25, 25],
       [25, 25, 25, 25],
       [25, 25, 25, 25]]), array([[20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20]])]


In [21]:
np.shape(result)

/Users/ruby/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(2,)